In [1]:
from setup_general import *

# setup data for classifier

In [2]:
data = train_est_prepared.copy()
rebalancing = False

features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

In [10]:
data.type.value_counts()

photo                                           2680
photo negative                                  1516
design/drawing/sketch                            567
plan                                             558
archaeological find                              549
poster                                           537
letter                                           434
book                                             366
document                                         325
manuscript                                       316
graphics                                         163
music sheet                                      149
script, song/vocal music                         141
coin                                             140
seal/imprint                                     133
digital image                                    130
postcard                                         108
small print                                       80
magazines                                     

# single rf classifier

In [6]:
#function to have resamplers resample to specific number of samples per class
def by_num(y, min_samples):
    b = Counter(y).values()
    a = Counter(y).keys()
    a = list(a)
    b = list(b)

    if min_samples > max(b):
        min_samples = max(b)

    for i in range(len(a)):
        if b[i] < min_samples :
            b[i] = min_samples
    return dict(zip(a, b))

In [9]:
from imblearn.pipeline import Pipeline

rfc = RandomForestClassifier(n_estimators=500, random_state=0)
skf = StratifiedKFold(n_splits=4)

val_acc = []

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    over = SMOTE(sampling_strategy=by_num(y_train_fold,100), k_neighbors=1,random_state=0)

    steps = [('over', over),('model', rfc)]

    pipe = Pipeline(steps=steps)
    pipe.fit(X_train_fold, y_train_fold)

    y_pred = rfc.predict(X_test_fold)
    val_acc.append(accuracy_score(y_test_fold, y_pred))

print(val_acc)


/home/till/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(


[0.8775510204081632, 0.8822157434402332, 0.8845481049562682, 0.882798833819242]


# sweeping

In [ ]:
project = 'rf'

# Define sweep config
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': 
    {
        'split': {'values': [2, 5, 10, 100]},
        'depth': {'values': [3, 6, 10, 50, 100, 1000]},
        'leaf': {'values': [2, 6, 10, 50, 100, 1000, 5000]},
        'estimators': {'values': [200, 500, 1000, 2000]},
        'features': {'values': [None, 'sqrt', 'log2']},

     }
}

# Initialize sweep by passing in config. (Optional) Provide a name of the project.
sweep_id = wandb.sweep(sweep=sweep_configuration, project=project)

def main():
    run = wandb.init(project=project)

    # note that we define values from `wandb.config` instead 
    # of defining hard values 
    split = wandb.config.split
    depth = wandb.config.depth
    leaf = wandb.config.leaf
    estimators = wandb.config.estimators
    feat = wandb.config.features
    

    # -------------------------- usual training code starts here  -------------------------------------
    
    rfc = RandomForestClassifier(n_estimators=estimators, max_depth=depth, min_samples_leaf=leaf, max_features=feat, min_samples_split=split, random_state=42)
    rfc.fit(X_train, y_train)

    y_pred = rfc.predict(X_test)
    val_acc = accuracy_score(y_test, y_pred)
    
    y_pred = rfc.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred)

    print(train_acc, val_acc)

    # -------------------------- ends here  -------------------------------------
    

    wandb.log({
      'train_acc': train_acc,
      'val_acc': val_acc,
    })

# Start sweep job.
wandb.agent(sweep_id, function=main)

# save model

In [5]:
pickle.dump(rfc, open('./models/rf/train_prep_full_best', 'wb'))
loaded_model = pickle.load(open('./models/rf/first_try', 'rb'))

# submission from model

In [ ]:
test_set = test_prep.drop('type', axis=1)
submission = pd.DataFrame({'id': test_set.index ,'type': rfc.predict(test_set)})
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())
submission.to_csv('submissions/some_rf_model.csv', index=False)